# Model and embeddings generation

In this tutorial, we will cover the generation of the Doc2Vec model for the hybrid-dictionary-ner approach. The aim is to produce embeddings for each RELISH and TREC publication.

# Prerequisites

1. Preprocessed tokens in .npy format or .tsv format. They can be generated following the preprocessing tutorial